In [1]:
## Importing the libraries
from numpy import genfromtxt
from time import time
from datetime import datetime
from sqlalchemy import Column, Integer, Float, Date, String, VARCHAR
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import csv, redis, json
import pandas as pd
import sys
import os
import pymongo


## Creating the Redis database
REDIS_HOST = 'localhost'

def read_csv_data(csv_file, ik, iv):
    with open(csv_file, encoding='utf-8') as csvf:
        csv_data = csv.reader(csvf)
        return [(r[ik], r[iv]) for r in csv_data]

def store_data(conn, data):
    for i in data:
        conn.setnx(i[0], i[1])
    return data

columns = (0, 1) 
data = read_csv_data('_hits_counter.csv', *columns)
conn = redis.Redis(REDIS_HOST)
json.dumps(store_data(conn, data))


## Defining and creating tables of the MongoDB database
def import_content1(filepath1):
    mng_client = pymongo.MongoClient('localhost', 27017)
    mng_db = mng_client['Mixed_MongoDB'] 
    collection_name = 'Product_catalogue' 
    db_cm = mng_db[collection_name]
    cdir = os.path.dirname('__file__')
    file_res = os.path.join(cdir, filepath1)
    
    fields = ['product_id','product_url','product_name','product_category_tree','retail_price','description','product_rating','product_specifications']
    data = pd.read_csv(file_res, usecols = fields)
    data_json = json.loads(data.to_json(orient='records'))
    db_cm.remove()
    db_cm.insert(data_json)
    
def import_content2(filepath2):
    mng_client = pymongo.MongoClient('localhost', 27017)
    mng_db = mng_client['Mixed_MongoDB'] 
    collection_name = 'Customer_details' 
    db_cm = mng_db[collection_name]
    cdir = os.path.dirname('__file__')
    file_res = os.path.join(cdir, filepath2)
    
    fields = ['customer_id','customer_zip_code_prefix','customer_city','customer_state']
    data = pd.read_csv(file_res, usecols = fields)
    data_json = json.loads(data.to_json(orient='records'))
    db_cm.remove()
    db_cm.insert(data_json)


## Importing data from the csv files
filepath1 = '/Users/admin/Downloads/BE_project-1.3/_product_catalogue.csv' 
import_content1(filepath1)

filepath2 = '/Users/admin/Downloads/BE_project-1.3/_customers_dataset.csv' 
import_content2(filepath2)


## Defining table of the SQLite database
Base = declarative_base()

class payment(Base):
    #Tell SQLAlchemy what the table name is and if there's any table-specific arguments it should know about
    __tablename__ = 'Payment'
    __table_args__ = {'sqlite_autoincrement': True}
    #tell SQLAlchemy the name of column and its attributes:
    customer_id = Column(VARCHAR(51))
    product_id = Column(VARCHAR(51))
    order_id = Column(VARCHAR(51), primary_key=True, nullable=False)
    payment_type = Column(VARCHAR(51))
    payment_installments = Column(Integer)
    payment_value = Column(Float)       

fields3 = ['customer_id','product_id','order_id','payment_type','payment_installments','payment_value']

file_name3 = '_order_payments_dataset.csv'
df3 = pd.read_csv(file_name3, usecols = fields3)

## Creating the SQLite database
engine = create_engine('sqlite:///Mixed_SQLite.db')
Base.metadata.create_all(engine)

df3.to_sql(con=engine, index_label='id', name=payment.__tablename__, if_exists='replace')

ModuleNotFoundError: No module named 'redis'